In [1]:
!pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch

print("PyTorch version:", torch.__version__)


PyTorch version: 2.1.0+cu118


In [3]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torchvision.models as models
import torchvision

from torchvision import transforms
import time
torch.manual_seed(17)

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print(device)

cuda


In [6]:
transform = transforms.Compose([ 
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
])

In [7]:
dataset = torchvision.datasets.ImageFolder('./tiny-imagenet-200/train', transform=transform)

In [8]:
len(dataset)

100000

In [9]:
#split the data
train_data, val_data, test_data = torch.utils.data.random_split(dataset, [80000, 10000, 10000])

In [10]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)

In [11]:
# Load pretrained ResNet-50 (Teacher Model)
teacher = models.resnet50(pretrained=True)

# Modify the final fully connected layer for 10 classes (CIFAR-10)
teacher.fc = nn.Linear(teacher.fc.in_features, 200)
# Move models to device
teacher = teacher.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:

model_path = 'best_teacher_model.pth'
# Load the model weights
teacher.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [13]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet18(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 200)
student = student.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [15]:
# Logits normalization function
def normalize(logit):
    mean = logit.mean(dim=-1, keepdim=True)
    stdv = logit.std(dim=-1, keepdim=True)
    return (logit - mean) / (1e-7 + stdv)


In [16]:
# CA-KLD Loss for Classification
def cakld_loss(student_logits, teacher_logits, beta_prob):
    # Forward KL (student || teacher)
    student_log_prob = F.log_softmax(student_logits, dim=1)
    teacher_prob = F.softmax(teacher_logits, dim=1)
    forward_kl = F.kl_div(student_log_prob, teacher_prob, reduction='batchmean')

    # Reverse KL (teacher || student)
    teacher_log_prob = F.log_softmax(teacher_logits, dim=1)
    student_prob = F.softmax(student_logits, dim=1)
    reverse_kl = F.kl_div(teacher_log_prob, student_prob, reduction='batchmean')

    # Combined KL loss
    kl_loss = beta_prob * reverse_kl + (1 - beta_prob) * forward_kl
    return kl_loss


In [17]:
def evaluate(model, test_loader, device):
    model = model.to(device)  # Ensure model is on the correct device
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [18]:
def calculate_sparsity(model):
    total_zeros = 0
    total_params = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_zeros += torch.sum(param == 0).item()
            total_params += param.numel()
    return total_zeros / total_params

In [19]:
import torch
import time
def measure_inference_time(model, test_loader, num_runs=5):
    device = torch.device('cpu')
    model.eval()
    model.to(device)

    # Warm-up (one batch to avoid startup cost)
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            _ = model(inputs)
            break

    total_time = 0
    total_images = 0

    with torch.no_grad():
        for _ in range(num_runs):
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                start_time = time.time()
                _ = model(inputs)
                end_time = time.time()

                total_time += (end_time - start_time)
                total_images += batch_size

    avg_time_per_image = total_time / total_images
    return avg_time_per_image


In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def calculate_model_size(model, filename="temp.pth"):
    torch.save(model.state_dict(), filename)
    size = os.path.getsize(filename) / (1024 * 1024)  # Size in MB
    os.remove(filename)
    return size

def compare_model_sizes(teacher, student, pruned_student):
    # Count parameters
    teacher_params = count_parameters(teacher)
    student_params = count_parameters(student)
    pruned_params = count_parameters(pruned_student)
    
    # Calculate disk size
    teacher_size = calculate_model_size(teacher, "teacher.pth")
    student_size = calculate_model_size(student, "student.pth")
    pruned_size = calculate_model_size(pruned_student, "pruned_student.pth")
    
    # Print comparison
    print("\n--- Model Size Comparison ---")
    print(f"Teacher Model: {teacher_params} parameters, {teacher_size:.2f} MB")
    print(f"Student Model (Before Pruning): {student_params} parameters, {student_size:.2f} MB")
    print(f"Student Model (After Pruning): {pruned_params} parameters, {pruned_size:.2f} MB")
    
    # Calculate compression ratio
    compression_ratio = student_size / pruned_size
    print(f"\nCompression Ratio: {compression_ratio:.2f}x")

In [21]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, patience=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0  # Counter for early stopping
    
    for epoch in range(epochs):
        print(epoch)
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Evaluate on the validation set
        val_accuracy = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        # Early stopping logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_teacher_model.pth')  # Save the best model
            print(f" New best model saved with validation accuracy: {best_val_accuracy:.2f}%")
        else:
            patience_counter += 1
            print(f" No improvement in validation accuracy ({patience_counter}/{patience})")
            
            # Stop training if no improvement for 'patience' epochs
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered! No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    model.load_state_dict(torch.load('best_teacher_model.pth'))
    print("\nLoading the best model for final evaluation.")
    
    # Evaluate on the test set
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Test Accuracy with Best Model: {test_accuracy:.2f}%")
    
    return model



In [22]:
# # Fine-tune the teacher model
# teacher = train_model(teacher, train_loader, val_loader, epochs=200, lr=0.001, patience=5)

In [23]:
def compute_gradient_importance(
    teacher, student, data_loader, device, temperature=4.0, alpha=0.5, beta_prob=0.5, accumulation_epochs=3
):
    importance_scores = {}

    # Initialize importance score storage for conv layer weights only
    for name, param in student.named_parameters():
        if 'weight' in name and len(param.shape) == 4:  # Conv weights only
            importance_scores[name] = torch.zeros_like(param.data, device=device)

    teacher.to(device).eval()
    student.to(device).train()

    # Add momentum for gradient accumulation smoothing
    momentum = 0.9  # Controls exponential moving average
    accumulated_batches = 0  # Track for bias correction

    for epoch in range(accumulation_epochs):
        print(f"Accumulation Epoch {epoch+1}/{accumulation_epochs}")
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            student.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            student_logits_temp = student_logits / temperature
            teacher_logits_temp = teacher_logits / temperature

            # Compute losses
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)
            loss = alpha * distillation_loss + (1 - alpha) * ce_loss

            # Modified backward propagation
            loss.backward()

            # Accumulate importance scores with parameter-gradient product
            accumulated_batches += 1
            for name, param in student.named_parameters():
                if name in importance_scores and param.grad is not None:
                    # Key modification: Use parameter-gradient product magnitude
                    grad_product = (param.data * param.grad).abs_()
                    
                    # Exponential moving average with bias correction
                    if accumulated_batches == 1:
                        importance_scores[name] = grad_product
                    else:
                        importance_scores[name] = momentum * importance_scores[name] + (1 - momentum) * grad_product

    # Apply bias correction for EMA
    for name in importance_scores:
        importance_scores[name] /= (1 - momentum**accumulated_batches)

    return importance_scores

In [24]:
def gradient_based_global_prune(model, importance_scores, prune_ratio=0.95):
    all_scores = torch.cat([score.flatten() for score in importance_scores.values()])
    threshold = torch.topk(all_scores, k=int(prune_ratio * all_scores.numel()), largest=False)[0][-1]

    for name, param in model.named_parameters():
        if name in importance_scores:
            mask = (importance_scores[name] > threshold).float()
            param.data.mul_(mask)

    return model


In [25]:
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def retrain_with_sparsity(student, train_loader, val_loader, epochs=5, save_path="retrained_student_model.pt", patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks AND zero momentum buffers for pruned weights
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:  # Consider only conv layers
            mask = (param != 0).float().to(device)
            masks[name] = mask
            # Zero momentum buffers for pruned weights
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    student = student.to(device)
    best_val_acc = 0.0
    best_model = None
    patience_counter = 0  # Counter for early stopping

    # 2. Add gradient clipping to prevent NaN
    max_grad_norm = 1.0

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = student(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Apply masks to gradients
            for name, param in student.named_parameters():
                if name in masks:
                    param.grad.data *= masks[name]

            # Gradient clipping before optimizer step
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_grad_norm)

            optimizer.step()

            # Reapply masks and update momentum buffers
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation phase
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total

        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = student.state_dict()
            torch.save(best_model, save_path)
            patience_counter = 0  # Reset patience counter
            print(f"New best model saved with Val Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break  # Stop training

        # Print results
        sparsity = calculate_sparsity(student)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.2f}% | Sparsity: {sparsity*100:.2f}%\n")

    print(f"Best Validation Accuracy: {best_val_acc:.2f}% | Best Model Saved at: {save_path}")
    return student

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

# KD training with CA-KLD loss and mask-based momentum handling
def retrain_with_KD(teacher, student, train_loader, val_loader, epochs=50,
                    temperature=5.0, alpha=0.5, beta_prob=0.5, patience=5,
                    save_path="student_before_pruning.pth"):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks and zero momentum buffers
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:
            mask = (param != 0).float().to(device)
            masks[name] = mask
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    teacher = teacher.to(device).eval()
    student = student.to(device)

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Apply temperature
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)


            # CA-KLD loss
            kd_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)

            loss = alpha * kd_loss + (1 - alpha) * ce_loss
            loss.backward()
            optimizer.step()

            # Reapply masks and update momentum
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total
        sparsity = calculate_sparsity(student) * 100.0  # Assuming this function is defined elsewhere

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Sparsity: {sparsity:.2f}%")

        # Early stopping logic
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Restore and save best model
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")
    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [27]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Training function with KD + CA-KLD and logits normalization
def train_kd_pruning(teacher, student, train_loader, val_loader, epochs=50, temperature=5.0, alpha=0.5,
                     beta_prob=0.5, patience=5, save_path="student_before_pruning.pth"):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    teacher = teacher.to(device)
    student = student.to(device)
    teacher.eval()  # Freeze teacher

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)

            # CA-KLD loss (normalized logits)
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)

            # Cross-entropy loss
            ground_truth_loss = F.cross_entropy(student_logits, labels)

            # Combined loss
            loss = alpha * distillation_loss + (1 - alpha) * ground_truth_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation accuracy
        val_acc = evaluate(student, val_loader, device)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Load best model state and save
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")

    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [28]:

# student = train_kd_pruning(
#     teacher, student, train_loader, val_loader,
#     epochs=50, temperature=5.0, alpha=0.5,beta_prob=0.5, patience=5,save_path="student_before_pruning.pth"
# )


In [29]:
# # Calculate sparsity
# sparsity = calculate_sparsity(student)
# print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

# teacher_accuracy = evaluate(teacher, test_loader, device)
# student_accuracy = evaluate(student, test_loader, device)
# print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
# print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

## 95% Sparsity

In [30]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9591)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)

# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 47s
Pruning the model
Total Time take to prune the model scores: 0m 0s
Sparsity: 95.00%


In [31]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 0.39%


In [32]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 36.9409 | Train Acc: 41.50% | Val Loss: 2.4584 | Val Acc: 46.15% | Sparsity: 95.00%
Epoch 2/50 | Train Loss: 27.2914 | Train Acc: 56.82% | Val Loss: 2.3406 | Val Acc: 50.10% | Sparsity: 95.00%
Epoch 3/50 | Train Loss: 24.0906 | Train Acc: 62.32% | Val Loss: 2.2530 | Val Acc: 51.23% | Sparsity: 95.00%
Epoch 4/50 | Train Loss: 21.9335 | Train Acc: 65.82% | Val Loss: 2.2120 | Val Acc: 51.87% | Sparsity: 95.00%
Epoch 5/50 | Train Loss: 20.3339 | Train Acc: 68.63% | Val Loss: 2.1852 | Val Acc: 52.80% | Sparsity: 95.00%
Epoch 6/50 | Train Loss: 18.9518 | Train Acc: 71.10% | Val Loss: 2.2063 | Val Acc: 52.59% | Sparsity: 95.00%
Epoch 7/50 | Train Loss: 17.7228 | Train Acc: 73.48% | Val Loss: 2.1727 | Val Acc: 53.76% | Sparsity: 95.00%
Epoch 8/50 | Train Loss: 16.6533 | Train Acc: 75.51% | Val Loss: 2.1819 | Val Acc: 52.97% | Sparsity: 95.00%
Epoch 9/50 | Train Loss: 15.6222 | Train Acc: 77.37% | Val Loss: 2.2579 | Val Acc: 51.95% | Sparsity: 95.00%
Epoch 10/50 | Train

In [33]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 52.71%


In [34]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9591)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)

# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 47s
Pruning the model
Total Time take to prune the model scores: 0m 0s
Sparsity: 95.00%


In [35]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 15.2713 | Train Acc: 36.55% | Val Loss: 2.6191 | Val Acc: 46.40% | Sparsity: 95.00%
Epoch 2/50 | Train Loss: 10.8494 | Train Acc: 55.72% | Val Loss: 2.3887 | Val Acc: 50.77% | Sparsity: 95.00%
Epoch 4/50 | Train Loss: 8.4077 | Train Acc: 66.35% | Val Loss: 2.2726 | Val Acc: 52.89% | Sparsity: 95.00%
Epoch 5/50 | Train Loss: 7.6992 | Train Acc: 69.39% | Val Loss: 2.2664 | Val Acc: 53.22% | Sparsity: 95.00%
Epoch 6/50 | Train Loss: 7.1420 | Train Acc: 72.17% | Val Loss: 2.2418 | Val Acc: 54.25% | Sparsity: 95.00%
Epoch 7/50 | Train Loss: 6.6904 | Train Acc: 74.21% | Val Loss: 2.2508 | Val Acc: 53.37% | Sparsity: 95.00%
Epoch 8/50 | Train Loss: 6.2644 | Train Acc: 76.57% | Val Loss: 2.2364 | Val Acc: 53.69% | Sparsity: 95.00%
Epoch 9/50 | Train Loss: 5.9006 | Train Acc: 78.15% | Val Loss: 2.2362 | Val Acc: 53.78% | Sparsity: 95.00%
Epoch 10/50 | Train Loss: 5.5764 | Train Acc: 79.99% | Val Loss: 2.2521 | Val Acc: 53.44% | Sparsity: 95.00%
Epoch 11/50 | Train Loss:

In [36]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 53.42%


# 90% of Sparsity

In [37]:
model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9086)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)

# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 49s
Pruning the model
Total Time take to prune the model scores: 0m 0s
Sparsity: 90.00%


In [38]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 0.39%


In [39]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 28.5414 | Train Acc: 55.29% | Val Loss: 2.3303 | Val Acc: 49.19% | Sparsity: 90.00%
Epoch 2/50 | Train Loss: 20.2194 | Train Acc: 68.33% | Val Loss: 2.2452 | Val Acc: 51.84% | Sparsity: 90.00%
Epoch 3/50 | Train Loss: 16.9516 | Train Acc: 73.95% | Val Loss: 2.1748 | Val Acc: 53.63% | Sparsity: 90.00%
Epoch 4/50 | Train Loss: 14.6006 | Train Acc: 78.23% | Val Loss: 2.1853 | Val Acc: 53.43% | Sparsity: 90.00%
Epoch 5/50 | Train Loss: 12.7298 | Train Acc: 81.45% | Val Loss: 2.1937 | Val Acc: 53.59% | Sparsity: 90.00%
Epoch 6/50 | Train Loss: 11.1733 | Train Acc: 84.37% | Val Loss: 2.2592 | Val Acc: 52.43% | Sparsity: 90.00%
Epoch 7/50 | Train Loss: 9.7565 | Train Acc: 87.03% | Val Loss: 2.1980 | Val Acc: 53.50% | Sparsity: 90.00%
Epoch 8/50 | Train Loss: 8.5787 | Train Acc: 89.14% | Val Loss: 2.2490 | Val Acc: 53.19% | Sparsity: 90.00%
Early stopping triggered at epoch 8. No improvement for 5 epochs.
Student model saved before pruning at: pruned_student_retrain_KD

In [40]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 53.48%


In [41]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9086)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)

# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 48s
Pruning the model
Total Time take to prune the model scores: 0m 0s
Sparsity: 90.00%


In [42]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 10.8144 | Train Acc: 56.79% | Val Loss: 2.1713 | Val Acc: 52.65% | Sparsity: 90.00%
Epoch 2/50 | Train Loss: 6.8405 | Train Acc: 73.26% | Val Loss: 2.1310 | Val Acc: 54.21% | Sparsity: 90.00%
Epoch 3/50 | Train Loss: 5.3792 | Train Acc: 79.69% | Val Loss: 2.1194 | Val Acc: 54.21% | Sparsity: 90.00%
Epoch 4/50 | Train Loss: 4.4374 | Train Acc: 84.09% | Val Loss: 2.1968 | Val Acc: 53.89% | Sparsity: 90.00%
Epoch 5/50 | Train Loss: 3.7585 | Train Acc: 87.44% | Val Loss: 2.2200 | Val Acc: 53.78% | Sparsity: 90.00%
Epoch 6/50 | Train Loss: 3.2066 | Train Acc: 89.88% | Val Loss: 2.2598 | Val Acc: 53.01% | Sparsity: 90.00%
Epoch 7/50 | Train Loss: 2.7612 | Train Acc: 91.83% | Val Loss: 2.2818 | Val Acc: 52.51% | Sparsity: 90.00%
Early stopping triggered at epoch 7. No improvement for 5 epochs.
Student model saved before pruning at: pruned_student_retrain_KD_90%.pth
Total Training Time: 9m 41s
Retraining completed in 9.68 minutes (581.07 seconds)


In [43]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 53.11%


# 79% of Sparsity

In [44]:
model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.7976)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)

# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 51s
Pruning the model
Total Time take to prune the model scores: 0m 0s
Sparsity: 79.00%


In [45]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 1.08%


In [46]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 19.1969 | Train Acc: 70.84% | Val Loss: 2.4012 | Val Acc: 51.69% | Sparsity: 79.00%
Epoch 2/50 | Train Loss: 12.9146 | Train Acc: 80.53% | Val Loss: 2.4798 | Val Acc: 51.58% | Sparsity: 79.00%
Epoch 3/50 | Train Loss: 9.9591 | Train Acc: 85.89% | Val Loss: 2.3443 | Val Acc: 52.77% | Sparsity: 79.00%
Epoch 4/50 | Train Loss: 7.5880 | Train Acc: 90.27% | Val Loss: 2.3046 | Val Acc: 53.30% | Sparsity: 79.00%
Epoch 5/50 | Train Loss: 5.7091 | Train Acc: 93.62% | Val Loss: 2.3368 | Val Acc: 54.27% | Sparsity: 79.00%
Epoch 6/50 | Train Loss: 4.2278 | Train Acc: 96.19% | Val Loss: 2.3686 | Val Acc: 53.73% | Sparsity: 79.00%
Epoch 7/50 | Train Loss: 3.1465 | Train Acc: 98.01% | Val Loss: 2.2722 | Val Acc: 54.57% | Sparsity: 79.00%
Epoch 8/50 | Train Loss: 2.2735 | Train Acc: 99.12% | Val Loss: 2.2369 | Val Acc: 54.88% | Sparsity: 79.00%
Epoch 9/50 | Train Loss: 1.7174 | Train Acc: 99.66% | Val Loss: 2.1463 | Val Acc: 56.28% | Sparsity: 79.00%
Epoch 10/50 | Train Loss: 

In [47]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 57.90%


In [48]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.7976)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)

# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 51s
Pruning the model
Total Time take to prune the model scores: 0m 0s
Sparsity: 79.00%


In [49]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 4.7144 | Train Acc: 84.08% | Val Loss: 2.2461 | Val Acc: 54.54% | Sparsity: 79.00%
Epoch 2/50 | Train Loss: 2.0159 | Train Acc: 94.22% | Val Loss: 2.2882 | Val Acc: 53.70% | Sparsity: 79.00%
Epoch 3/50 | Train Loss: 1.2318 | Train Acc: 97.60% | Val Loss: 2.2803 | Val Acc: 54.35% | Sparsity: 79.00%
Epoch 4/50 | Train Loss: 0.8494 | Train Acc: 99.03% | Val Loss: 2.2488 | Val Acc: 54.67% | Sparsity: 79.00%
Epoch 5/50 | Train Loss: 0.6444 | Train Acc: 99.62% | Val Loss: 2.2272 | Val Acc: 55.15% | Sparsity: 79.00%
Epoch 6/50 | Train Loss: 0.5304 | Train Acc: 99.82% | Val Loss: 2.1774 | Val Acc: 55.53% | Sparsity: 79.00%
Epoch 7/50 | Train Loss: 0.4571 | Train Acc: 99.92% | Val Loss: 2.1250 | Val Acc: 55.97% | Sparsity: 79.00%
Epoch 8/50 | Train Loss: 0.4177 | Train Acc: 99.95% | Val Loss: 2.1399 | Val Acc: 55.83% | Sparsity: 79.00%
Epoch 9/50 | Train Loss: 0.3850 | Train Acc: 99.97% | Val Loss: 2.1290 | Val Acc: 55.81% | Sparsity: 79.00%
Epoch 10/50 | Train Loss: 0.

In [50]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 56.31%


## 59% Sparsity

In [51]:
model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.5957)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)

# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 51s
Pruning the model
Total Time take to prune the model scores: 0m 0s
Sparsity: 59.00%


In [52]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 12.51%


In [53]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 10.8772 | Train Acc: 85.06% | Val Loss: 2.7107 | Val Acc: 50.19% | Sparsity: 59.00%
Epoch 2/50 | Train Loss: 8.4976 | Train Acc: 88.31% | Val Loss: 2.5311 | Val Acc: 52.88% | Sparsity: 59.00%
Epoch 3/50 | Train Loss: 5.5691 | Train Acc: 93.62% | Val Loss: 2.4397 | Val Acc: 53.94% | Sparsity: 59.00%
Epoch 4/50 | Train Loss: 3.6060 | Train Acc: 96.98% | Val Loss: 2.3783 | Val Acc: 54.13% | Sparsity: 59.00%
Epoch 5/50 | Train Loss: 2.2970 | Train Acc: 98.96% | Val Loss: 2.2178 | Val Acc: 57.24% | Sparsity: 59.00%
Epoch 6/50 | Train Loss: 1.4924 | Train Acc: 99.76% | Val Loss: 2.1148 | Val Acc: 57.98% | Sparsity: 59.00%
Epoch 7/50 | Train Loss: 1.0939 | Train Acc: 99.94% | Val Loss: 2.0455 | Val Acc: 58.29% | Sparsity: 59.00%
Epoch 8/50 | Train Loss: 0.9033 | Train Acc: 99.96% | Val Loss: 1.9949 | Val Acc: 59.27% | Sparsity: 59.00%
Epoch 9/50 | Train Loss: 0.7891 | Train Acc: 99.98% | Val Loss: 1.9966 | Val Acc: 59.16% | Sparsity: 59.00%
Epoch 10/50 | Train Loss: 0

In [54]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 59.96%


In [55]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.5957)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)

# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 51s
Pruning the model
Total Time take to prune the model scores: 0m 0s
Sparsity: 59.00%


In [56]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.8030 | Train Acc: 99.40% | Val Loss: 2.1658 | Val Acc: 56.98% | Sparsity: 59.00%
Epoch 2/50 | Train Loss: 0.3499 | Train Acc: 99.97% | Val Loss: 2.1201 | Val Acc: 57.61% | Sparsity: 59.00%
Epoch 3/50 | Train Loss: 0.2843 | Train Acc: 99.98% | Val Loss: 2.0959 | Val Acc: 58.05% | Sparsity: 59.00%
Epoch 4/50 | Train Loss: 0.2573 | Train Acc: 99.98% | Val Loss: 2.0692 | Val Acc: 58.32% | Sparsity: 59.00%
Epoch 5/50 | Train Loss: 0.2411 | Train Acc: 99.98% | Val Loss: 2.0588 | Val Acc: 58.41% | Sparsity: 59.00%
Epoch 6/50 | Train Loss: 0.2270 | Train Acc: 99.98% | Val Loss: 2.0510 | Val Acc: 58.33% | Sparsity: 59.00%
Epoch 7/50 | Train Loss: 0.2159 | Train Acc: 99.98% | Val Loss: 2.0394 | Val Acc: 58.28% | Sparsity: 59.00%
Epoch 8/50 | Train Loss: 0.2079 | Train Acc: 99.98% | Val Loss: 2.0425 | Val Acc: 58.74% | Sparsity: 59.00%
Epoch 9/50 | Train Loss: 0.2031 | Train Acc: 99.98% | Val Loss: 2.0378 | Val Acc: 58.48% | Sparsity: 59.00%
Epoch 10/50 | Train Loss: 0.

In [57]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 58.86%


## 36% Sparsity

In [58]:
model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.3635)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)

# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 51s
Pruning the model
Total Time take to prune the model scores: 0m 0s
Sparsity: 36.00%


In [59]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 50.88%


In [60]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.6485 | Train Acc: 99.98% | Val Loss: 1.9882 | Val Acc: 60.14% | Sparsity: 36.00%
Epoch 2/50 | Train Loss: 0.5483 | Train Acc: 99.98% | Val Loss: 1.9897 | Val Acc: 59.66% | Sparsity: 36.00%
Epoch 3/50 | Train Loss: 0.5084 | Train Acc: 99.98% | Val Loss: 1.9606 | Val Acc: 60.08% | Sparsity: 36.00%
Epoch 4/50 | Train Loss: 0.4771 | Train Acc: 99.98% | Val Loss: 1.9673 | Val Acc: 60.44% | Sparsity: 36.00%
Epoch 5/50 | Train Loss: 0.4528 | Train Acc: 99.98% | Val Loss: 1.9477 | Val Acc: 60.43% | Sparsity: 36.00%
Epoch 6/50 | Train Loss: 0.4351 | Train Acc: 99.98% | Val Loss: 1.9537 | Val Acc: 60.66% | Sparsity: 36.00%
Epoch 7/50 | Train Loss: 0.4232 | Train Acc: 99.98% | Val Loss: 1.9386 | Val Acc: 60.79% | Sparsity: 36.00%
Epoch 8/50 | Train Loss: 0.4100 | Train Acc: 99.98% | Val Loss: 1.9299 | Val Acc: 60.85% | Sparsity: 36.00%
Epoch 9/50 | Train Loss: 0.3988 | Train Acc: 99.98% | Val Loss: 1.9487 | Val Acc: 60.65% | Sparsity: 36.00%
Epoch 10/50 | Train Loss: 0.

In [61]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 61.27%


In [62]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.3635)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)

# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity: {sparsity * 100:.2f}%")

Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 3m 49s
Pruning the model
Total Time take to prune the model scores: 0m 0s
Sparsity: 36.00%


In [63]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.2257 | Train Acc: 99.98% | Val Loss: 1.9545 | Val Acc: 60.26% | Sparsity: 36.00%
Epoch 2/50 | Train Loss: 0.1781 | Train Acc: 99.98% | Val Loss: 1.9416 | Val Acc: 60.28% | Sparsity: 36.00%
Epoch 3/50 | Train Loss: 0.1669 | Train Acc: 99.98% | Val Loss: 1.9332 | Val Acc: 60.53% | Sparsity: 36.00%
Epoch 4/50 | Train Loss: 0.1605 | Train Acc: 99.98% | Val Loss: 1.9279 | Val Acc: 60.53% | Sparsity: 36.00%
Epoch 5/50 | Train Loss: 0.1549 | Train Acc: 99.98% | Val Loss: 1.9274 | Val Acc: 60.52% | Sparsity: 36.00%
Epoch 6/50 | Train Loss: 0.1509 | Train Acc: 99.98% | Val Loss: 1.9140 | Val Acc: 60.45% | Sparsity: 36.00%
Epoch 7/50 | Train Loss: 0.1485 | Train Acc: 99.98% | Val Loss: 1.9159 | Val Acc: 60.52% | Sparsity: 36.00%
Epoch 8/50 | Train Loss: 0.1452 | Train Acc: 99.98% | Val Loss: 1.9166 | Val Acc: 60.64% | Sparsity: 36.00%
Epoch 9/50 | Train Loss: 0.1426 | Train Acc: 99.98% | Val Loss: 1.9212 | Val Acc: 60.62% | Sparsity: 36.00%
Epoch 10/50 | Train Loss: 0.

In [64]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 61.35%
